In [1]:
ls

cotacaotreino_win_dol.xlsx            modelo_final.ipynb
Download_dados_relatorio_final.ipynb


In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import seaborn
import statsmodels.api as sm
from scipy.stats import zscore
from statsmodels import regression
from statsmodels.tsa.stattools import coint

In [3]:
data = pd.read_excel('cotacaotreino_win_dol.xlsx')

In [4]:
data.head()

,time,open WIN$,high WIN$,low WIN$,close WIN$,volumeWIN$,open DOL$,high DOL$,low DOL$,close DOL$,volumeDOL$
0,2016-12-21 13:25:00,70695,70852,70689,70815,8192,3829.959,3831.106,3829.959,3831.106,645
1,2016-12-21 13:30:00,70815,70815,70477,70544,22378,3831.106,3832.252,3830.532,3831.106,1160
2,2016-12-21 13:35:00,70538,70664,70501,70622,8733,3831.106,3831.106,3829.959,3830.532,860
3,2016-12-21 13:40:00,70628,70664,70520,70556,6383,3831.106,3831.106,3828.239,3828.812,885
4,2016-12-21 13:45:00,70556,70652,70532,70592,5014,3829.386,3831.106,3829.386,3831.106,1295


In [47]:
data = data[['close WIN$','close DOL$']]

In [48]:
data

,close WIN$,close DOL$
0,70815,3831.106
1,70544,3831.106
2,70622,3830.532
3,70556,3828.812
4,70592,3831.106
...,...,...
89243,80275,5492.138
89244,80315,5492.639
89245,80180,5494.142
89246,80140,5498.650


In [185]:
class Fica_rico():
    def __init__(self, dados, tickers):
        self.dados = dados[:5000]
        #seleciona os tickers
        self.t1 = tickers[0]
        self.t2 = tickers[1]
        #calcula variações
        self.s1 = self.dados[self.t1].pct_change()[1:]
        self.s2 = self.dados[self.t2].pct_change()[1:]
        self.dados = self.dados[1:]
        
        self.posicionado = 0 #está neutro
        self.result_1 = 0
        self.result_2 = 0
        self.fin = 0 #valor financeiro
    
    #funcao para calcular o zscores do spread entre os pares
    def calcula_zscore_spread(self,s1,s2):
        s1 = sm.add_constant(s1)
        results = sm.OLS(s2,s1).fit()
        s1 = s1[self.t1]
        b = results.params[self.t1]
        spread = s2 - (b*s1)
        zs = zscore(spread)[-1]
        return zs
    
    #condicoes para entrar/encerrar uma operacao 
    def condicoes(self,zs,preco_s1,preco_s2, razao):
        if zs >= 1.1 and self.posicionado == 0:
            self.posicionado = 1 #está posicionado
            self.q = razao #para deixa igual o valor de s1 e s2

            self.result_1 -= preco_s1 #debito valor das acoes
            self.result_2 += preco_s2*self.q #credito valor da s2 (tá short)
            
        elif (zs < .08) and self.posicionado == 1:
            self.posicionado = 0 #está neutro
            self.result_1 += preco_s1 #encerra posicao short
            self.result_2 -= preco_s2*self.q #encerra posicao long
            resultado_trade = self.result_1 + self.result_2
            self.fin += resultado_trade
            self.result_1, self.result_2 = 0,0 #reseta operacao
            
    def opera(self,):
        c = 0
        for index in range(45,len(self.dados)):
            s1 = self.s1[c:index]
            s2 = self.s2[c:index]
            preco_s1 = self.dados[self.t1][self.dados.index == index].values[0]
            preco_s2 = self.dados[self.t2][self.dados.index == index].values[0]
            q12 = preco_s1/preco_s2
        
            zs = self.calcula_zscore_spread(s1,s2)
            self.condicoes(zs,preco_s1,preco_s2,razao=q12)

In [181]:
tickers = ['close WIN$','close DOL$']

In [182]:
modelo = Fica_rico(data,tickers)

In [183]:
modelo.opera()

In [184]:
modelo.fin

4350.377454843954